In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
import numpy as np

In [3]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [4]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


/Users/ima/tasks/1_2021/qiskit/terra/qiskit/quantum_info/states/statevector.py:444: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  for z, x, coeff in zip(oper.table.Z, oper.table.X, oper.coeffs)


## ExpectationValue class

In [5]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [6]:
from qiskit.providers.aer import AerSimulator

backend = AerSimulator()

In [7]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5], shos=1000)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueResult(value=1.84765625, variance=11.072105407714844, confidence_interval=(1.6896409070896277, 2.0056715929103723))

In [8]:
# pre-binding

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueResult(value=1.9453125, variance=10.988372802734375, confidence_interval=(1.7880903253842175, 2.1025346746157823))

In [9]:
# multi prameters

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]], shots=3000)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueArrayResult(values=array([1.87333333, 0.13066667]), variances=array([11.07357689, 10.33122844]), confidence_intervals=array([[1.78082897, 1.9658377 ],
       [0.03751989, 0.22381344]]))

In [10]:
# can pass ndarray
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]), shots=8192)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueArrayResult(values=array([1.8527832 , 0.15209961]), variances=array([11.13814348, 10.56786078]), confidence_intervals=array([[1.79654633, 1.90902008],
       [0.09517763, 0.20902159]]))

### Exact simulation by SaveExpectationValueVariance

In [11]:
from qiskit.evaluators import ExactExpectationValue

In [12]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.8420921273153708, variance=6.432763522806668, confidence_interval=None)

In [13]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]))

ExpectationValueArrayResult(values=array([1.84209213, 0.15611897]), variances=array([6.43276352, 5.35681431]), confidence_intervals=array([None, None], dtype=object))

### Transpiled Circuits

In [14]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.set_transpile_options(initial_layout=[3, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
    print()
print(expval.evaluate([0, 1, 1, 2, 3, 5]))

global phase: π
                                                                »
ancilla_0 -> 0 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 1 ─────────────────────────────────────────────────»
               ┌────┐┌──────────────┐┌────┐┌────────┐┌───┐┌────┐»
      q_1 -> 2 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤└─┬─┘├────┤»
      q_0 -> 3 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘     └────┘»
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c4: 2/═════════════════════════════════════════════════»
                                                                »
«                                                                          »
«ancilla_0 -> 0 ─────────────────────────────────

### Transpile options and Run options

In [15]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=100_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.1833734963853253, variance=12.776249008490854, confidence_interval=(1.165449013944694, 1.2012979788259566))

In [16]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.168212890625, variance=12.758465230464935, confidence_interval=(1.1056568133433762, 1.2307689679066238))

In [17]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=1.1866666666666665, variance=12.703377777777778, confidence_interval=(0.8607444915023681, 1.512588841830965))

### Composite Evaluator

In [18]:
from qiskit.evaluators import JointEvaluator

In [19]:
len(expval.transpiled_circuits)

3

In [20]:
joint_evaluator = JointEvaluator([expval, expval, expval])  # can be different evaluator

In [21]:
len(joint_evaluator.transpiled_circuits)  # 3 × 3

9

In [22]:
joint_evaluator.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13]])

CompositeResult(items=[ExpectationValueResult(value=1.19091796875, variance=12.800391912460327, confidence_interval=(1.1282004069837166, 1.2536355305162834)), ExpectationValueResult(value=1.19091796875, variance=12.800391912460327, confidence_interval=(1.1282004069837166, 1.2536355305162834)), ExpectationValueResult(value=1.19091796875, variance=12.800391912460327, confidence_interval=(1.1282004069837166, 1.2536355305162834))])

### Readout error mitigation

In [23]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit = ReadoutErrorMitigation(
    backend, mitigation="tensored", refresh=600, shots=2000, mit_pattern=[[0], [1]]
)
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_mit = PauliExpectationValue(ansatz, observable, backend=mit)
shots = 2000
print(
    f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)
print(
    f"w/  mitigation shots={shots}, result={expval_mit.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)

w/o mitigation shots=2000, result=ExpectationValueResult(value=1.251, variance=12.780410999999999, confidence_interval=(1.1241444207813454, 1.3778555792186544))
w/  mitigation shots=2000, result=ExpectationValueResult(value=1.3008475460143756, variance=12.538071538402928, confidence_interval=(1.1756829842352605, 1.4260121077934906))


### Gradient of expectation value

In [24]:
from qiskit.evaluators.expectation_value.expectation_value_gradient import FiniteDiffGradient, ParameterShiftGradient

In [25]:
# exact_expval = ExactExpectationValue(ansatz, observable, backend=AerSimulator())
# exact_findiff = FiniteDiffGradient(exact_expval, 1e-8)
# print(f"fin diff of exact {exact_findiff.evaluate([0, 1, 1, 2, 3, 5]).values}")

shots = 2000
findiff = FiniteDiffGradient(expval_raw, 1e-1)
paramshift = ParameterShiftGradient(expval_raw)
print(f"fin diff w/o mit {findiff.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")
print(f"param shift w/o mit {paramshift.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")

findiff = FiniteDiffGradient(expval_mit, 1e-1)
paramshift = ParameterShiftGradient(expval_mit)
print(f"fin diff w/  mit {findiff.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")
print(f"param shift w/  mit {paramshift.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")


fin diff w/o mit [-1.4  -1.75 -0.25 -0.87 -0.54 -0.93]
param shift w/o mit [0.099  1.403  0.7745 1.3005 0.941  1.4915]
fin diff w/  mit [-1.29058112  0.21874482 -0.70796302 -0.8627791  -1.15390334  1.86511967]
param shift w/  mit [0.16088677 1.56708761 0.78777228 1.40973804 1.21876092 1.75961842]


### Scipy optimizer

In [26]:
from scipy.optimize import minimize
shot = 10000
backend = AerSimulator()
expval = PauliExpectationValue(ansatz, observable, backend=backend)
paramshift = ParameterShiftGradient(expval)
# this may take a long time...
result = minimize(lambda x: expval_mit.evaluate(x, shots=shots).value, np.zeros(6),
                  jac=lambda x: paramshift.evaluate(x, shots=shots).values)
print(result)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


      fun: -4.609263861759417
 hess_inv: array([[ 0.20515557, -0.23576343, -0.0484571 ,  0.20094939,  0.03524248,
        -0.14764537],
       [-0.23576343,  0.75165856,  0.43825389, -0.75034371, -0.77853314,
         0.13959995],
       [-0.0484571 ,  0.43825389,  0.48110665, -0.61393319, -0.7352592 ,
         0.06838818],
       [ 0.20094939, -0.75034371, -0.61393319,  1.18642541,  1.14208052,
        -0.19590744],
       [ 0.03524248, -0.77853314, -0.7352592 ,  1.14208052,  1.60009024,
         0.01585804],
       [-0.14764537,  0.13959995,  0.06838818, -0.19590744,  0.01585804,
         0.30897136]])
      jac: array([ 0.004 ,  0.3395, -0.1235,  0.31  ,  0.34  , -0.295 ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 50
      nit: 9
     njev: 32
   status: 2
  success: False
        x: array([-1.62739621,  1.81345777,  3.21381643, -1.46110082, -2.33609977,
        0.96455941])
